In [8]:
# testing Excel and Pivot for GINI MUP from Joanna

# copy the file:
import shutil  # for file manipulation
import os
import win32com.client as win32
from ok_sap_script import *
import time
from datetime import date, timedelta, datetime, timezone
import pyautogui
from functools import wraps
import logging # custom logging created
from pathlib import Path # working with / // or \


class MyLogger:
    # Define custom log levels
    Custom1 = 15 # number are for levels having different number behind (10 for Debug and 50 for Error, I suppose)
    Custom2 = 45

    def __init__(self, log_file_path, log_level=logging.INFO):
        '''Define path, define minimal log level, call setup function'''
        self.receiver_log = "oleksandr.komarov@zoetis.com" # my name in case of log failure
        self.log_file_path = log_file_path
        self.log_level = log_level
        self.logger = self._setup_logger()

    def _setup_logger(self): # _ means that this method is to be used inside this class only (even though nothing prevants you from using it anyway)
        logger = logging.getLogger(self.log_file_path)
        logger.setLevel(self.log_level)
        self._add_file_handler(logger) # how to handle log .txt file
        self._add_console_handler(logger) # how logs are displayed in the console

        return logger

    def _add_file_handler(self, logger):
        file_formatter = self._get_file_formatter()
        with open(self.log_file_path, mode='w') as log_file:
            file_handler = logging.FileHandler(self.log_file_path, mode='w')
            file_handler.setFormatter(file_formatter)
            logger.addHandler(file_handler)

    def _add_console_handler(self, logger):
        console_formatter = self._get_console_formatter()
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(console_formatter)
        logger.addHandler(console_handler)

    def _get_file_formatter(self):
        if self.log_level == logging.INFO:
            return logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        else:
            return logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s: error occured at line %(lineno)d', datefmt='%Y-%m-%d %H:%M:%S')

    def _get_console_formatter(self):
        if self.log_level <= logging.INFO:
            return logging.Formatter('%(levelname)s - %(message)s')
        elif self.log_level <= logging.ERROR:
            return logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s: error occured at line %(lineno)d', datefmt='%Y-%m-%d %H:%M:%S')
        else:
            if self.log_level == MyLogger.Custom1:
                return logging.Formatter('%(levelname)s - %(message)s')
            elif self.log_level == MyLogger.Custom2:
                return logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
            else:
                return logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    def log(self, message, level=logging.INFO):
        '''Main function to write a log message'''
        self.logger.log(level, message)
        if level >= logging.ERROR:
            self.reason = message
            self._send_email_failed()

    def _send_email_failed(self):
        # send this failure by email
        self.outlook = win32.Dispatch('outlook.application')
        self.mail = self.outlook.CreateItem(0)
        self.mail.To = self.receiver_log
        self.mail.Subject = f"Control failure :("
        self.mail.Body = f"This is to inform you that one of the controls failed.\nReason {self.reason}\n.Please consult attached log file to know a reason."
        self.mail.Attachments.Add(f"{self.log_file_path}")
        #self.mail.Display(True)
        self.mail.Send()

class MainControl():
    '''Set variables and functions that are copied from control to control.
    By doing this we opt for more standartisation'''
    def __init__(self):
        # main variables
        self.curr_user = os.getlogin() # what user runs this code
        self.date_stamp = datetime.today().strftime("%d-%m-%Y") # datestamp to differentiate between runs
        self.receiver = "oleksandr.komarov@zoetis.com" # who will receive an email with control
        # paths
        self.generic_path = Path(f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/") # path to the folder, where controls are stored
        # SAP variables global
        self.environment = "ECC Production" # which environment to use
        # handling dates in SAP
        self.period = date.today().strftime("%#m") # use # to remove a leading 0 -> result 7 and not 07, 8 and not 08
        self.period_0 = date.today().strftime("%m") # print with leading 0 -> 07, 08, etc.
        self.year = datetime.today().year # get current year

    def __repr__(self) -> str: # how our control is seen for the users, who print it (more official)
        return (f"Control on {self.curr_user} machine") # an instance of this class will have this text if printed
    
    def __str__(self) -> str: # how our control is seen for the users, who print it (more user-friendly)
        return (f"Control by name {self.__class__.__name__} on {self.curr_user} machine") # an instance of this class will have this text if printed
    
    @staticmethod
    def sap_decorator(sap_function):
        '''Closing SAP sessions before and after running a script, entering tcode, performing logging and error handling'''
        @wraps(sap_function)
        def sap_wrapper(self, t_code_sap, *args, **kwargs):
            sap_close() # ensure there are no open session
            self.main_logger.log(level=logging.INFO, message="SAP is terminated prior function execution")
            open_sap = sap_open() # open new session
            self.main_logger.log(level=logging.INFO, message="New SAP session is opened")
            time.sleep(5) # make sure SAP opens up
            if open_sap == True:
                self.main_logger.log(level=logging.INFO, message=f"Trying to enter {self.environment} environment")
                self.session = sap_logon(environment=self.environment, client=1)
                self.main_logger.log(level=logging.INFO, message=f"Trying to execute {t_code_sap} code in SAP")
                sap_code(tcode=t_code_sap, session=self.session)
                sap_function(self) # this is the main fuction to be decorated
            else:
                self.main_logger.log(level=logging.ERROR, message="SAP did not open")
            sap_close() # close SAP
            self.main_logger.log(level=logging.INFO, message="SAP is terminated after function execution")
        return sap_wrapper
    
    @staticmethod
    def timer_decorator(func_to_time):
        '''This one is used to track the execution time of any particular function'''
        import time
        @wraps(func_to_time)
        def time_wrapper(self, *args, **kwargs):
            t1 = time.time()
            func_to_time(self, *args, **kwargs)
            t2 = time.time() - t1
            self.main_logger.log(level=logging.INFO, message='{} run in: {} seconds'.format(func_to_time.__name__, round(t2,2)))
        return time_wrapper

    @staticmethod
    def excel_decorator(excel_function):
        '''Opening Excel, applying some standard parameters for runtime optimisation, etc.'''
        def excel_wrapper(self, *args, **kwargs):
            try:
                # default Excel runtime optimisation
                self.excel = win32.Dispatch("Excel.Application")
                self.excel.AskToUpdateLinks = False
                self.excel.DisplayAlerts = False
                self.excel.Visible = True
                self.excel.ScreenUpdating = False
                excel_function(self) # our main Excel function to run
                # no closing decorators as it differs from control to control
            except Exception as e:
                # return to a normal Excel, then close it
                self.excel.ScreenUpdating = True
                self.excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
                close_excel() # close excel
                self.main_logger.log(level=logging.ERROR, message=f'Error while using Excel or Excel-related functions, namely, {e}')
        return excel_wrapper
    
    def copy_file(self, file_from: str, file_to: str):
        '''Simple function to copy one file to a given location'''
        try:
            shutil.copy(file_from, file_to)
            self.main_logger.log(level=logging.INFO, message=f"File {file_from} is copied as {file_to}")
        except PermissionError as e:
            self.main_logger.log(level=logging.ERROR, message=f"Could not create a new file, the file might be open")
            os.system("taskkill /f /im  excel.exe")

    def take_screenshot(self, name:str, path:str):
        '''A simple function to take screenshots using pyautogui\n 
        It is not used in this class and purposed for inheritance\n
        Args:
            name (str): a file name, spare .png. 
            path (str): fr path, meaning / being used as separators.\n
        Example:
            take_screenshot(name=faglb03_scr_for_30-08-2023, path=fr'C:/Users/KOMAROVO/Desktop/Automations/CPT_TP/GIT/Screenshots/')\n
        Returns:
            None
            '''
        try: # try taking screenshot
            screenshot = pyautogui.screenshot() 
            screenshot.save(str(Path(f'{path}/{name}.png')))
            self.main_logger.log(level=logging.INFO, message=f"Screenshot is taken with a name of {name}.png under location {path}")
        except Exception as e: 
            self.main_logger.log(level=logging.ERROR, message=f"Failed to take screenshot {name} for a reason {e}")

    def add_screenshots(self, worksheet, list_screenshots, picture_path):
        '''This function will add screenshot or screenshots into Excel worksheet'''
        try:
            # remove old screenshots
            self.pictures = worksheet.Pictures()
            for pic in self.pictures:
                pic.Delete()
            self.main_logger.log(level=logging.INFO, message=f"Old screenshot removed from {worksheet}")
            # add screenshots
            self.left, self.top, self.width, self.height = 0, 30, 792, 534
            for screenshot in list_screenshots:
                self.picture_filename = screenshot
                self.full_picture_path = str(Path(f"{picture_path}/{self.picture_filename}.png"))
                # insert a new screenshot with given parameters
                self.picture = worksheet.Shapes.AddPicture(self.full_picture_path, LinkToFile=False, SaveWithDocument=True, Left=self.left, Top=self.top, Width=self.width, Height=self.height)
                self.top += self.height # the following one will be put below the current one
                self.main_logger.log(level=logging.INFO, message=f"Screenshot with a name {self.picture_filename}.png is added")
        except Exception as e:
            self.main_logger.log(level=logging.ERROR, message=f"No screenshots are found or {e}")

    def send_email(self, subject:str = "Automatic email", body:str="Please consult the attachment"):
        '''Send email'''
        self.subject = subject
        self.body = body
        # send by email
        self.outlook = win32.Dispatch('outlook.application')
        self.mail = self.outlook.CreateItem(0)
        self.mail.To = self.receiver
        self.mail.Subject = f"{self.subject} for {self.date_stamp}"
        self.mail.Body = self.body
        try:
            self.mail.Attachments.Add(f"{self.new_file_path}")
        except:
            self.main_logger.log(level=logging.ERROR, message=f"No attachment found under location {self.new_file_path}")
        #self.mail.Display(True)
        self.mail.Send()
        self.main_logger.log(level=logging.INFO, message=f"Email sent to {self.receiver}")

# this part is to be adjusted according to the most recent approach
class GINI_MUP():
    def __init__(self):
        self.curr_user = os.getlogin()
        self.period = date.today().strftime("%#m") # use # to remove a leading 0 -> result 7 and not 07, 8 and not 08
        self.formula_history = "=CONCATENATE(B2,G2)"

        # set paths
        self.final_path_to_prev = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GINI_MUP/GINI_MUP_model.xlsb" # GINI_template
        self.new_file_path = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GINI_MUP/GINI open items AP09.xlsb"

        self.final_path_to_history = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GINI_MUP/historical costs AP 9 2023 - LE 2941.xlsx"
        # same for SAP
        self.final_path_to_sap = f"C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GINI_MUP/Output/GINI_extract_SAP.XLSX"
        self.sap_extract_path = f"C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\GINI_MUP\\Output\\"
        self.sap_extract_name = "GINI_extract_SAP.XLSX"

        # SAP variables
        self.environment = "ECC Production"
        self.t_code_sap = "fbl3n"
        self.variant_sap = "MUP_AUTO"

        # Excel
        self.cocds = ['2941', '2942', '2946', '2951']

    def sap_part(self):
        # fbl3n part
        # ensure there are no open sessions
        sap_close()
        # open new session
        open_sap = sap_open()
        time.sleep(5)
        if open_sap == True:
            session = sap_logon(environment=self.environment, client=1)
            sap_code(tcode=self.t_code_sap, session=session)
            try:
                sap_variant(session=session, var_to_use=self.variant_sap)
            except: 
                sap_variant(session=session, var_to_use=self.variant_sap, version=2) # in our case, V2 is the most probable scenario
            # change date
            today = datetime.today()
            first = today.replace(day=1)
            last_month = (first - timedelta(days=1)).strftime("%d.%m.%Y")
            session.findById("wnd[0]/usr/ctxtPA_STIDA").text = last_month
            # run the report
            sap_run(session=session)
            sap_extract(session=session, extr_path=self.sap_extract_path, extr_name=self.sap_extract_name)
            sap_close()
            close_excel()

    def copy_file(self):
        try:
            shutil.copy(self.final_path_to_prev, self.new_file_path)
        except PermissionError as e:
            print("The file may be opened")
            os.system("taskkill /f /im  excel.exe")
    # open created Excel file and perform the following manipulations
        try:
            # default Excel runtime optimisation
            excel = win32.Dispatch("Excel.Application")
            excel.AskToUpdateLinks = False
            excel.DisplayAlerts = False
            excel.Visible = True
            excel.ScreenUpdating = False
            # open new file and disable calculations
            self.new_file = excel.Workbooks.Open(self.new_file_path)
            excel.Application.Calculation = (-4135)  # to set xlCalculationManual # Workbook needs to be opened
    # other steps
                # update historical cost sheet
                    # open historical cost file
            self.historical_file = excel.Workbooks.Open(self.final_path_to_history)
                    # find last row of this file
            self.historical_file_ws = self.historical_file.Worksheets(1)
            self.last_row_historical_file_ws = self.historical_file_ws.Cells(1, 1).End(-4121).Row
                    # rename historical costs AP{self.period}
            self.historical_costs_ws = self.new_file.Sheets("hc")
            self.historical_costs_ws.Name = f"hc"
                    # clear range in historical costs AP{self.period}
            self.historical_costs_ws.Range("A:AF").ClearContents()
                    # copy paste data from the historical workbook into costs AP{self.period}
            self.historical_file_ws.Range(f"A1:AE{self.last_row_historical_file_ws}").SpecialCells(12).Copy(Destination=self.historical_costs_ws.Range(f"B1"))
                    # apply formula in the column A - =CONCATENATE(B2,G2)
            self.historical_costs_ws.Range(f"A2:A{self.last_row_historical_file_ws}").Formula = "=CONCATENATE(B2,G2)"
                    # add cost per 1
            self.historical_costs_ws.Range(f"U1").Value = "Price per 1"
            self.historical_costs_ws.Range(f"U2:U{self.last_row_historical_file_ws}").Formula = "=T2/V2"
                # update GINI data sheet
                    # work with GINI extract from SAP
            self.sap_gini_file = excel.Workbooks.Open(self.final_path_to_sap) # open gini extract
            self.sap_gini_file_ws = self.sap_gini_file.Worksheets(1) # get default first sheet
            self.last_row_sap_gini_file_ws = self.sap_gini_file_ws.Cells(1, 1).End(-4121).Row # get last row
                    # copy paste this data to GINI data sheet
                        # clear previous data
            self.gini_data_ws = self.new_file.Sheets(f"GINI data")
            self.gini_data_ws.Range("A:AA").ClearContents()
                        # paste new data # we do not need to copy final row with totals
            self.sap_gini_file_ws.Range(f"A1:AA{self.last_row_sap_gini_file_ws-1}").SpecialCells(12).Copy(Destination=self.gini_data_ws.Range(f"A1"))

    # work with the pivot table
                # define pivot table
            self.pivot_ws = self.new_file.Sheets(f"Pivot")
                # refresh pivot
            self.pivot_itself = self.pivot_ws.PivotTables("PivotTable3")
            # update data source
                # select GINI data, where the database is located
            self.gini_data_ws = self.new_file.Sheets(f"GINI data")
                # define borders (we know it goes from A to AA but have to define last row)
            self.last_row_gini_data = self.gini_data_ws.Cells(1, 1).End(-4121).Row
            print("Data ", self.last_row_gini_data)
                # create pivot cash
            self.pivot_cache = self.new_file.PivotCaches().Create(SourceType=1, SourceData=self.gini_data_ws.Range(f"A1:AA{self.last_row_gini_data}"))
                # apply pivot cash
            self.pivot_itself.ChangePivotCache(self.pivot_cache)
                # refresh pivot, so that new data is applied
            self.pivot_ws.PivotTables("PivotTable3").PivotCache().Refresh()
                # where does this Pivot end?
            self.pivot_itself.ClearAllFilters() # otherwise the results might be compromised
            self.last_row_pivot = self.pivot_ws.Cells(4, 1).End(-4121).Row
                # add columns to Pivot
                    # clear contents
            self.pivot_ws.Range("I:J").ClearContents()
                    # column1
            self.pivot_ws.Range("I4").Value = "Qty in BUoM"
            self.pivot_ws.Range(f"I5:I{self.last_row_pivot-1}").Formula = "=E5/H5" # -1 to exclude the last row with totals
                    # column2
            self.pivot_ws.Range("J4").Value = "BUoM"
            self.pivot_ws.Range(f"J5:J{self.last_row_pivot-1}").Formula = f"=VLOOKUP(G5,'hc'!A:E,5,FALSE)"
                # check all items
            for item in self.pivot_itself.PivotFields("Company Code").PivotItems():
                print(item)
                    # set filter to the pivot
                self.pivot_itself.ClearAllFilters()
                # reference -> https://learn.microsoft.com/en-us/office/vba/api/Excel.XlPivotFilterType
                self.pivot_itself.PivotFields("Company Code").PivotFilters.Add2(17, None, item) # xlCaptionBeginsWith -> 17
                    # define new last row
                self.last_row_pivot = self.pivot_ws.Cells(4, 1).End(-4121).Row
                    # clear respective worksheet, which corresponds to "item" value
                self.cocd_ws = self.new_file.Sheets(f"{item}") # define the sheet, use "" because otherwise it will not find correct sheet
                self.cocd_ws.Range("A:G").ClearContents()
                    # copy pivot table data to the respective sheet
                self.pivot_ws.Range(f"A4:J{self.last_row_pivot}").SpecialCells(12).Copy(Destination=self.cocd_ws.Range(f"A1"))
            # clear filters from pivot
            self.pivot_itself.ClearAllFilters()
            # close all other files
            self.historical_file.Close()
            self.sap_gini_file.Close()
            self.new_file.Save()

            # back to default
            excel.ScreenUpdating = True
            excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
        except Exception as e:
            excel.ScreenUpdating = True
            excel.Application.Calculation = -4105  # to set xlCalculationAutomatic
            print(f"something went wrong, namely\n{e}")

    def sap_faglb03(self):
        today = datetime.today()
        year = today.year
        # ensure there are no open sessions
        sap_close()
        # open new session
        open_sap = sap_open()
        time.sleep(5)
        if open_sap == True:
            session = sap_logon(environment=self.environment, client=1)
            sap_code(tcode="faglb03", session=session)
            for cocd in self.cocds:
                # populate values
                session.findById("wnd[0]/usr/ctxtRACCT-LOW").text = "101401"
                session.findById("wnd[0]/usr/ctxtRBUKRS-LOW").text = cocd
                session.findById("wnd[0]/usr/txtRYEAR").text = year
                # run the report
                sap_run(session=session)
                # take balance
                balance_value = session.findById("wnd[0]/usr/cntlFDBL_BALANCE_CONTAINER/shellcont/shell").GetCellValue(f"{self.period}", "BALANCE")
                session.findById("wnd[0]/usr/cntlFDBL_BALANCE_CONTAINER/shellcont/shell").setCurrentCell(f"{self.period}","BALANCE")
                try:
                    screenshot_first_page = pyautogui.screenshot()
                    screenshot_first_page.save(fr'C:/Users/{self.curr_user}/Desktop/Automations/CPT_TP/GINI_MUP/Screenshots/Balance_for_{cocd}_{datetime.today().strftime("%d-%m-%Y")}.png')
                except Exception as e: print(e)
                # go to the previous screen
                session.findById("wnd[0]").sendVKey(3)
            sap_close()
    
    def paste_imgs(self):
        # default Excel runtime optimisation
        excel = win32.Dispatch("Excel.Application")
        excel.AskToUpdateLinks = False
        excel.DisplayAlerts = False
        excel.Visible = True
        excel.ScreenUpdating = False
        # open new file and disable calculations
        self.new_file = excel.Workbooks.Open(self.new_file_path)
        excel.Application.Calculation = (-4135)  # to set xlCalculationManual # Workbook needs to be opened
        for code in self.cocds:
            self.cocd_ws = self.new_file.Sheets(f"{code}")
            pictures = self.cocd_ws.Pictures()
            for pic in pictures:
                pic.Delete()
            picture_filename = f'Balance_for_{code}_{datetime.today().strftime("%d-%m-%Y")}.png'
            picture_path = fr'C:\\Users\\{self.curr_user}\\Desktop\\Automations\\CPT_TP\\GINI_MUP\\Screenshots\\Balance_for_{code}_{datetime.today().strftime("%d-%m-%Y")}.png'
            print(picture_path)
            # Insert a new picture with given parameters
            left = 580
            top = 10
            width = 840
            height = 640

            picture = self.cocd_ws.Shapes.AddPicture(picture_path, LinkToFile=False, SaveWithDocument=True, Left=left, Top=top, Width=width, Height=height)
        # back to default
        excel.ScreenUpdating = True
        excel.Application.Calculation = -4105  # to set xlCalculationAutomatic

control = GINI_MUP()
control.sap_part()
control.copy_file()
control.sap_faglb03()
control.paste_imgs()




Data  2550
2941
2946
C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\GINI_MUP\\Screenshots\\Balance_for_2941_05-09-2023.png
C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\GINI_MUP\\Screenshots\\Balance_for_2942_05-09-2023.png
C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\GINI_MUP\\Screenshots\\Balance_for_2946_05-09-2023.png
C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\GINI_MUP\\Screenshots\\Balance_for_2951_05-09-2023.png


In [2]:
# This function is for testing material price determination setting in SAP
from ok_sap_script import *
import win32com.client as win32
import time
import os
from datetime import datetime

def sap_part():
    # outlook
    receiver = "oleksandr.komarov@zoetis.com"
    curr_user = os.getlogin()
    extract_path = f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Price_determination/Output"
    extract_name = f"PD-{datetime.today().strftime('%d-%m-%Y')}.xlsx"

    environment = "ECC (QE3)"
    t_code_sap = "se16"
    table_sap = "mbew"
    variant_sap = "MBEW_PD"
    variant_sap2 = "/MBEW_NB"
    # ensure there are no open sessions
    close_sap = sap_close()
    # open new session
    open_sap = sap_open()
    time.sleep(5)
    if open_sap == True:
        session = sap_logon(environment=environment, client=1)
        sap_code(tcode=t_code_sap, session=session)
        # enter mbew
        session.findById("wnd[0]/usr/ctxtDATABROWSE-TABLENAME").text = table_sap
        session.findById("wnd[0]").sendVKey(0)
        session.findById("wnd[0]/usr/ctxtI1-LOW").text = "10000000"
        session.findById("wnd[0]/usr/ctxtI1-HIGH").text = "19999999"
        session.findById("wnd[0]/usr/txtMAX_SEL").text = "10000000"
        session.findById("wnd[0]").sendVKey(8)

        # select layout
        sap_layout(session=session, layout=variant_sap)
        # check if there is no output
        mbew_body = session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")
        rows = mbew_body.rowCount
        if rows !=0:
            print("There is some data. Export")
            # extract
            sap_extract(session=session, extr_path=extract_path, extr_name=extract_name)
            close_excel()
            # send by email
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = receiver
            mail.Subject = f"Automatic email for price determination control"
            mail.Body = f"Please consult the attachment"
            mail.Attachments.Add(f"{extract_path}/{extract_name}")
            #mail.Display(True)
            mail.Send()
            sap_close()
        else:
            print("There is no data. End here")
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = receiver
            mail.Subject = f"Automatic email for price determination control"
            mail.Body = f"There are no materials to be corrected"
            #mail.Display(True)
            mail.Send()
            sap_close()
       


sap_part()

There is some data. Export


In [52]:
# This function is for testing TP montly emails in SAP
# Test SAP part

from ok_sap_script import *
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
import win32com.client as win32

curr_user = os.getlogin()
pi01_path = f"pi01-{datetime.today().strftime('%d-%m-%Y')}.xlsx"
ziv1_path = f"ziv1-{datetime.today().strftime('%d-%m-%Y')}.xlsx"

def sap_extract_me():   
    first_day = datetime.today().replace(day=1).strftime("%d.%m.%Y")
    environment = "ECC (QE3)"
    t_code_sap = "ZP2M_OTBD_SP_FL_NONF"
    variant_flash = "FLASH_ME"
    variant_non_flash = "NON_FLASH_ME" 
    

    # ensure there are no open sessions
    close_sap = sap_close()
    # open new session
    open_sap = sap_open()
    time.sleep(5)
    if open_sap == True:
        session = sap_logon(environment=environment, client=1)
        sap_code(tcode=t_code_sap, session=session)
        try:
            sap_variant(session=session, var_to_use=variant_flash)
        except: 
            sap_variant(session=session, var_to_use=variant_flash, version=2) # in our case, V2 is the most probable scenario

        session.findById("wnd[0]/usr/ctxtS_DATBI-LOW").text = first_day
        # run the report
        sap_run(session=session)
        sap_extract(session=session, extr_path="C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Files_extracted\\", extr_name=pi01_path)
        close_sap = sap_close()
    # Do the same but for non-flash
        # open new session
    open_sap = sap_open()
    time.sleep(5)
    if open_sap == True:
        session = sap_logon(environment=environment, client=1)
        sap_code(tcode=t_code_sap, session=session)
        try:
            sap_variant(session=session, var_to_use=variant_non_flash, version=1)
        except: 
            sap_variant(session=session, var_to_use=variant_non_flash, version=2) # in our case, V2 is the most probable scenario

        session.findById("wnd[0]/usr/ctxtS_DATB1-LOW").text = first_day # this one differs
        # Run the report
        sap_run(session=session)
        sap_extract(session=session, extr_path="C:\\Users\\KOMAROVO\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Files_extracted", extr_name=ziv1_path)
        close_sap = sap_close()
        close_excel()

def excel_rework_me():
    # flash
    #close_excel()
    global list_LEs
    list_LEs = []
    # grab with pandas, loop through, populate separete files in Files_spit, attach to email later
    df_pi01 = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_extracted/{pi01_path}")
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="flash_copy")
    for index, row in df_email_contacts.iterrows():
        df_pi01_split = df_pi01[df_pi01['Sales Organization'] == row['LE']]
        if df_pi01_split.empty: # if no record for particular market exist
            pass
        else:
            save_to = f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_split/{row['LE']}-{pi01_path}"
            df_pi01_split.to_excel(save_to, index=False)
            list_LEs.append(row['LE'])
            send_emails_me_flash(save_to, row['LE'])
    # non flash
    # do the same for ziv1 file
    df_ziv1 = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_extracted/{ziv1_path}")
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="non_flash_copy")
    for index, row in df_email_contacts.iterrows():
        df_pi01_split = df_ziv1[df_ziv1['Customer Number'] == row['Customer LE']]
        if df_pi01_split.empty: # if no record for particular market exist
            pass
        else:
            save_to = f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Output/Files_split/{row['LE']}-{ziv1_path}"
            df_pi01_split.to_excel(save_to, index=False)
            list_LEs.append(row['LE'])
            send_emails_me_non_flash(save_to, row['LE'])
    save_last_x_emails()
    
def save_last_x_emails():
    number_emails = len(list_LEs)
    # save files
    outlook = win32.Dispatch('outlook.application')
    mapi = outlook.GetNameSpace("MAPI")
    # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
    folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
    items= folder.Items
    items.Sort("[ReceivedTime]", Descending=True)
    msgs = items.GetFirst()
    msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\{msgs.Subject}.msg")
    for _ in range(number_emails - 1):
        msgs = items.GetNext()
        msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\{msgs.Subject}.msg")
    print(msgs)


def send_emails_me_flash(save_path, cocd):
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="flash_copy")
    for index, row in df_email_contacts.iterrows():
        if row['LE'] == cocd:
            #--Send file by email:
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = row['Emails2']
            mail.Subject = f"Automatic email for {row['LE']}"
            mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
            mail.Attachments.Add(save_path)
            #mail.Display(True)
            #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
            mail.Send()
            # time.sleep(3) # give enough time for the mail to appear in the mailbox
            # mapi = outlook.GetNameSpace("MAPI")
            # # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
            # folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
            # items= folder.Items
            # items.Sort("[ReceivedTime]", Descending=False)
            # msgs = items.GetLast()
            # msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
            # print(msgs)
        else: pass

def send_emails_me_non_flash(save_path, cocd):
    # same for the non flash
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="non_flash_copy")
    for index, row in df_email_contacts.iterrows():
        if row['LE'] == cocd:
            #--Send file by email:
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)
            mail.To = row['Emails2']
            mail.Subject = f"Automatic email for {row['LE']}"
            mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
            mail.Attachments.Add(save_path)
            #mail.Display(True)
            #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
            mail.Send()
            # time.sleep(3) # give enough time for the mail to appear in the mailbox
            # mapi = outlook.GetNameSpace("MAPI")
            # # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
            # folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
            # items= folder.Items
            # items.Sort("[ReceivedTime]", Descending=False)
            # msgs = items.GetLast()
            # msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
            # print(msgs) 
        else: pass

def send_emails_me_original(save_path):
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="flash_copy")
    for index, row in df_email_contacts.iterrows():
        print(row['LE'], row['Emails2'])
        #--Send file by email:
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = row['Emails2']
        mail.Subject = f"Automatic email for {row['LE']}"
        mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
        mail.Attachments.Add(save_path)
        #mail.Display(True)
        #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
        mail.Send()
        time.sleep(2) # give enough time for the mail to appear in the mailbox
        mapi = outlook.GetNameSpace("MAPI")
        # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
        folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
        items= folder.Items
        items.Sort("[ReceivedTime]", Descending=False)
        msgs = items.GetLast()
        msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
        print(msgs)
    # same for the non flash
    df_email_contacts = pd.read_excel(f"C:/Users/{curr_user}/Desktop/Automations/CPT_TP/Monthly_emails/Contacts_emails_ME.xlsx", sheet_name="non_flash_copy")
    for index, row in df_email_contacts.iterrows():
        print(row['LE'], row['Emails2'])
        #--Send file by email:
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = row['Emails2']
        mail.Subject = f"Automatic email for {row['LE']}"
        mail.Body = f"Please find attached a file with TP extract for your Cocd {row['LE']}"
        #mail.Attachments.Add(entry_path.get())
        #mail.Display(True)
        #mail.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Emails\\Email to {row['LE']}.msg") # this one will save the mails in the unsaved state. And if put below mail.Send() will crush, for the item is already sent
        mail.Send()
        time.sleep(2) # give enough time for the mail to appear in the mailbox
        mapi = outlook.GetNameSpace("MAPI")
        # 5 is for Sent items folder, 6 is for Inbox folder, more here: https://learn.microsoft.com/en-us/office/vba/api/outlook.oldefaultfolders
        folder = mapi.GetDefaultFolder(5)#.folders("example_folder") # default folder is defined via mapi (Messaging Application Programming Interface)
        items= folder.Items
        items.Sort("[ReceivedTime]", Descending=False)
        msgs = items.GetLast()
        msgs.SaveAs(f"C:\\Users\\{curr_user}\\Desktop\\Automations\\CPT_TP\\Monthly_emails\\Output\\Emails\\Email to {row['LE']}.msg")
        print(msgs)
        
sap_extract_me()
excel_rework_me()

Automatic email for 7927
